In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OrdinalEncoder
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import mlflow
import mlflow.sklearn
from sklearn.pipeline import Pipeline

# Chargement des données
chemin_dossier = "C:/Users/paulm/Documents/Projet 7/Projet7withCSV/data/"
df = pd.read_csv(os.path.join(chemin_dossier, 'processed_data.csv'))

# Vérifier si SK_ID_CURR est présente et correcte avant prétraitement
if 'SK_ID_CURR' in df.columns:
    print("SK_ID_CURR est présente avant prétraitement.")
    print(df['SK_ID_CURR'].head())
else:
    raise ValueError("SK_ID_CURR n'est pas présente dans les données d'origine.")

# Extraire SK_ID_CURR pour une utilisation ultérieure
sk_id_curr = df['SK_ID_CURR']

# Sélection des colonnes catégorielles
colonnes_categorielles = df.select_dtypes(include='object').columns.tolist()

# Affichage du nombre de valeurs uniques dans chaque colonne catégorielle
for col in colonnes_categorielles:
    nb_valeurs_uniques = df[col].nunique()
    print(f"Colonne '{col}' contient {nb_valeurs_uniques} valeurs uniques.")

# Suppression des colonnes avec trop de valeurs uniques
colonnes_a_supprimer = ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']
df.drop(columns=colonnes_a_supprimer, inplace=True)

# Identifier les colonnes numériques
colonnes_numeriques = df.select_dtypes(include=[np.number])

# Identifier les colonnes contenant des valeurs infinies
colonnes_infinies = colonnes_numeriques.columns.to_series()[np.isinf(colonnes_numeriques).any()]

# Affichage des colonnes avec des valeurs infinies
print("Colonnes avec des valeurs infinies :")
print(colonnes_infinies)

# Suppression des lignes contenant des valeurs infinies
lignes_avec_infinis = df.index[np.isinf(colonnes_numeriques).any(axis=1)]
df.drop(lignes_avec_infinis, inplace=True)

# Vérification
print("\nReste-t-il des valeurs infinies ?")
reste_infinis = np.isinf(colonnes_numeriques).any().any()
print(reste_infinis)

# Si des valeurs infinies restent, les remplacer par NaN
if reste_infinis:
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    print("\nLes valeurs infinies ont été remplacées par NaN.")

# Retirer SK_ID_CURR avant encodage
df.drop(columns=['SK_ID_CURR'], inplace=True)

# Pipeline de prétraitement
preprocessor = Pipeline(steps=[
    ('drop_columns', 'drop'),  # Ajouter étape de suppression de colonnes
    ('inf_to_nan', 'inf'),  # Ajouter étape de gestion des infinies
    ('encoder', OrdinalEncoder()),  # Encodage des variables catégorielles
    ('imputer', SimpleImputer(strategy='mean'))  # Imputation des valeurs manquantes
])

# Préparer les données
X = df.drop(columns=['TARGET'])
y = df['TARGET']

# Appliquer le prétraitement
X_preprocessed = preprocessor.fit_transform(X)

# Ajouter SK_ID_CURR de nouveau après encodage
df_encoded = pd.DataFrame(X_preprocessed, columns=X.columns)
df_encoded['SK_ID_CURR'] = sk_id_curr.values

# Vérifier si SK_ID_CURR est présente et correcte après encodage
if 'SK_ID_CURR' in df_encoded.columns:
    print("SK_ID_CURR est présente après encodage.")
    print(df_encoded['SK_ID_CURR'].head())
else:
    raise ValueError("SK_ID_CURR n'est pas présente après encodage.")

# Séparation des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df_encoded, y, test_size=0.2, random_state=42)

# Vérifier si SK_ID_CURR est présente et correcte après séparation
if 'SK_ID_CURR' in X_train.columns and 'SK_ID_CURR' in X_test.columns:
    print("SK_ID_CURR est présente après séparation.")
    print(X_train['SK_ID_CURR'].head())
    print(X_test['SK_ID_CURR'].head())
else:
    raise ValueError("SK_ID_CURR n'est pas présente après séparation.")

# Adresse de MLflow
mlflow_url = "http://127.0.0.1:5000"

# Coût d'un faux positif et d'un faux négatif
cost_fp = 1  
cost_fn = 10  

# Définir le modèle XGBoost avec ses paramètres
model = XGBClassifier(scale_pos_weight=10, random_state=42)
params = {'n_estimators': [50, 100], 'max_depth': [3, 6, 9]}

# Configurer l'expérience MLflow
mlflow.set_tracking_uri(mlflow_url)
mlflow.set_experiment("Credit Scoring Experiment")

# Échantillonner les données
sample_size = 0.1 
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, test_size=(1 - sample_size), random_state=42)
X_test_sample, _, y_test_sample, _ = train_test_split(X_test, y_test, test_size=(1 - sample_size), random_state=42)

# Entraînement du modèle XGBoost obtention des meilleurs paramètres
with mlflow.start_run(run_name="XGBoost"):
    print("Entraînement du modèle XGBoost...")

    grid_search = GridSearchCV(estimator=model, param_grid=params, scoring='f1', cv=3)
    grid_search.fit(X_train_sample, y_train_sample)

    # Loguer les paramètres dans MLflow
    mlflow.log_params(grid_search.best_params_)

    # Affichage des meilleurs paramètres et le score F1
    print(f"Meilleurs paramètres pour XGBoost: {grid_search.best_params_}")
    print(f"Score F1 moyen sur le jeu de validation: {grid_search.best_score_:.3f}")

    # Utilisation du meilleur modèle pour les prédictions
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test_sample)

    # Évaluation du modèle avec le meilleur seuil pour minimiser le coût métier
    y_prob = best_model.predict_proba(X_test_sample)[:, 1]
    thresholds = np.linspace(0, 1, 100)
    costs = []
    for threshold in thresholds:
        y_pred_thresholded = (y_prob > threshold).astype(int)
        fp = np.sum((y_pred_thresholded == 1) & (y_test_sample == 0)) * cost_fp
        fn = np.sum((y_pred_thresholded == 0) & (y_test_sample == 1)) * cost_fn
        total_cost = fp + fn
        costs.append(total_cost)

    best_threshold = thresholds[np.argmin(costs)]
    print(f"Meilleur seuil pour minimiser le coût métier avec XGBoost: {best_threshold}")

    y_pred_best_threshold = (y_prob > best_threshold).astype(int)

    # Affichage du rapport de classification et de la matrice de confusion
    classification_report_str = classification_report(y_test_sample, y_pred_best_threshold)
    confusion_matrix_str = confusion_matrix(y_test_sample, y_pred_best_threshold)

    print(f"Rapport de classification avec le meilleur seuil pour XGBoost:")
    print(classification_report_str)

    print(f"Matrice de confusion avec le meilleur seuil pour XGBoost:")
    print(confusion_matrix_str)
    print()

    # Loguer les métriques et les résultats dans MLflow
    mlflow.log_metric("best_score_f1", grid_search.best_score_)
    mlflow.log_metric("best_threshold", best_threshold)
    mlflow.log_text(classification_report_str, "classification_report.txt")
    mlflow.log_text(str(confusion_matrix_str), "confusion_matrix.txt")

    # Loguer le modèle
    model_path = 'model/xgboost_model.pkl'
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    mlflow.sklearn.log_model(best_model, "model")

    # Sauvegarder le prétraitement dans un fichier .pkl
    preprocessing_pipeline = {
        'drop_columns': colonnes_a_supprimer,
        'inf_to_nan': lignes_avec_infinis,
        'encoder': preprocessor.named_steps['encoder'],
        'imputer': preprocessor.named_steps['imputer']
    }

    preprocessing_path = 'preprocessing/preprocessing_pipeline.pkl'
    os.makedirs(os.path.dirname(preprocessing_path), exist_ok=True)
    joblib.dump(preprocessing_pipeline, preprocessing_path)


MemoryError: Unable to allocate 1.02 GiB for an array with shape (445, 307511) and data type float64

In [2]:
print(df_encoded['SK_ID_CURR'])

0              0.0
1              1.0
2              2.0
3              3.0
4              4.0
            ...   
307487    307487.0
307488    307488.0
307489    307489.0
307490    307490.0
307491    307491.0
Name: SK_ID_CURR, Length: 307492, dtype: float64
